In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Dense,Dropout,Conv2D,Conv1D,Flatten,MaxPool2D
import tensorflow as tf

In [2]:
data_path_and_labels = pd.read_csv("../mapped_data/ant-1.3.csv")
data_path_and_labels

,data_path,bugs
0,dataset/promise/defect data/ant-1.3/src/main/o...,0
1,dataset/promise/defect data/ant-1.3/src/main/o...,0
2,dataset/promise/defect data/ant-1.3/src/main/o...,0
3,dataset/promise/defect data/ant-1.3/src/main/o...,0
4,dataset/promise/defect data/ant-1.3/src/main/o...,0
...,...,...
119,dataset/promise/defect data/ant-1.3/src/main/o...,0
120,dataset/promise/defect data/ant-1.3/src/main/o...,0
121,dataset/promise/defect data/ant-1.3/src/main/o...,0
122,dataset/promise/defect data/ant-1.3/src/main/o...,0


In [3]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels

,data_path,bugs
0,../dataset/promise/defect data/ant-1.3/src/mai...,0
1,../dataset/promise/defect data/ant-1.3/src/mai...,0
2,../dataset/promise/defect data/ant-1.3/src/mai...,0
3,../dataset/promise/defect data/ant-1.3/src/mai...,0
4,../dataset/promise/defect data/ant-1.3/src/mai...,0
...,...,...
119,../dataset/promise/defect data/ant-1.3/src/mai...,0
120,../dataset/promise/defect data/ant-1.3/src/mai...,0
121,../dataset/promise/defect data/ant-1.3/src/mai...,0
122,../dataset/promise/defect data/ant-1.3/src/mai...,0


In [4]:
with open('/home/bavanya/Desktop/AST_text_regression/dataset/promise/defect data/ant-1.3/src/main/org/apache/tools/tar/TarBuffer.java.restrictedcontent.embed') as f:
    lines = f.read().splitlines()

In [5]:
embeddings = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    embeddings.append(lines)

In [6]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604
0,531.0,506.0,1966.0,1734.0,1956.0,1734.0,2724.0,1734.0,1592.0,1865.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,533.0,847.0,1058.0,847.0,1606.0,1734.0,1174.0,1734.0,1173.0,870.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2102.0,1865.0,1617.0,870.0,2037.0,1734.0,1824.0,1734.0,1865.0,2259.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,530.0,1734.0,136.0,1734.0,134.0,136.0,279.0,1701.0,416.0,2063.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,534.0,1734.0,328.0,1734.0,330.0,1734.0,329.0,847.0,1058.0,1734.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,468.0,2414.0,2119.0,504.0,1492.0,504.0,1931.0,847.0,802.0,504.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,2225.0,957.0,2119.0,2414.0,2119.0,504.0,2703.0,2119.0,959.0,2119.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,469.0,469.0,468.0,504.0,2581.0,1508.0,1678.0,2581.0,844.0,2262.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,361.0,504.0,1622.0,504.0,1314.0,578.0,2661.0,886.0,264.0,362.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604
0,0,531.0,506.0,1966.0,1734.0,1956.0,1734.0,2724.0,1734.0,1592.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,533.0,847.0,1058.0,847.0,1606.0,1734.0,1174.0,1734.0,1173.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2102.0,1865.0,1617.0,870.0,2037.0,1734.0,1824.0,1734.0,1865.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,530.0,1734.0,136.0,1734.0,134.0,136.0,279.0,1701.0,416.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,534.0,1734.0,328.0,1734.0,330.0,1734.0,329.0,847.0,1058.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,0,468.0,2414.0,2119.0,504.0,1492.0,504.0,1931.0,847.0,802.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,0,2225.0,957.0,2119.0,2414.0,2119.0,504.0,2703.0,2119.0,959.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,0,469.0,469.0,468.0,504.0,2581.0,1508.0,1678.0,2581.0,844.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,0,361.0,504.0,1622.0,504.0,1314.0,578.0,2661.0,886.0,264.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
train, test = train_test_split(data, test_size=0.2)

In [9]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604
112,0,253.0,504.0,1320.0,504.0,1319.0,1734.0,2140.0,1734.0,2137.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,0,225.0,1092.0,225.0,2489.0,1210.0,1678.0,1179.0,2618.0,844.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,0,2225.0,957.0,2119.0,2414.0,2119.0,504.0,2703.0,2119.0,959.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0,33.0,504.0,2558.0,1721.0,33.0,1028.0,2410.0,2595.0,2447.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0,322.0,1210.0,847.0,1854.0,357.0,435.0,923.0,435.0,2163.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,0,238.0,1734.0,774.0,1734.0,781.0,1734.0,1033.0,1734.0,2590.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,0,205.0,2419.0,2055.0,416.0,2418.0,1746.0,279.0,2420.0,1746.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0,509.0,504.0,1871.0,847.0,1125.0,2114.0,504.0,509.0,2114.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,2,36.0,1734.0,55.0,435.0,923.0,443.0,2163.0,847.0,2582.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604
59,0,578.0,578.0,1210.0,2703.0,147.0,1051.0,147.0,496.0,496.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,0,323.0,538.0,2610.0,847.0,1186.0,1734.0,1187.0,847.0,1184.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,0,600.0,225.0,1093.0,225.0,831.0,225.0,2793.0,504.0,2598.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110,0,372.0,504.0,1939.0,2367.0,1314.0,504.0,2453.0,2661.0,504.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0,607.0,225.0,2507.0,2437.0,2507.0,225.0,2507.0,2507.0,1434.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3,569.0,504.0,1148.0,538.0,2211.0,578.0,910.0,578.0,569.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119,0,468.0,2414.0,2119.0,504.0,1492.0,504.0,1931.0,847.0,802.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,0,538.0,2610.0,506.0,1828.0,1734.0,1962.0,2610.0,538.0,1962.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0,225.0,2503.0,225.0,1092.0,847.0,2242.0,2437.0,2503.0,504.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,0,498.0,68.0,2206.0,506.0,2259.0,506.0,498.0,1694.0,279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,1605

In [12]:
Y_train = pd.DataFrame(train[' bugs'], columns=[' bugs']).values
Y_test = pd.DataFrame(test[' bugs'], columns=[' bugs']).values

In [13]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [14]:
X_train = train.values
X_test = test.values

In [15]:
X_train

array([[ 253.,  504., 1320., ...,    0.,    0.,    0.],
       [ 225., 1092.,  225., ...,    0.,    0.,    0.],
       [2225.,  957., 2119., ...,    0.,    0.,    0.],
       ...,
       [ 509.,  504., 1871., ...,    0.,    0.,    0.],
       [  36., 1734.,   55., ...,    0.,    0.,    0.],
       [ 311., 2225.,  504., ...,    0.,    0.,    0.]])

In [16]:
# Preparing the data for the model.
X_train1 = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test1 = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [27]:
#Building the model
model = Sequential()

#add model layers
model.add(Conv2D(100, kernel_size=1, activation='relu',input_shape=input_shape))
#model.add(Conv2D(32, kernel_size=1, activation='relu'))
#model.add(Conv2D(16, kernel_size=1, activation='relu'))
    
    
#model.add(MaxPool2D(pool_size=(1,8)))
#model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))

#compile model using mse as the loss function
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])

In [28]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(X_train1, Y_train, epochs = 100)

Epoch 1/100
4/4 [==============================] - 0s 18ms/step - loss: 442.1724 - mse: 442.1724 - mae: 8.1078 - root_mean_squared_error: 21.0279 - mean_squared_logarithmic_error: 2.6326
Epoch 2/100
4/4 [==============================] - 0s 19ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 3/100
4/4 [==============================] - 0s 19ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 4/100
4/4 [==============================] - 0s 21ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 5/100
4/4 [==============================] - 0s 22ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 6/100
4/4 [==============================] - 0s 21ms/step - loss: 0.5354 - mse:

4/4 [==============================] - 0s 29ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 46/100
4/4 [==============================] - 0s 21ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 47/100
4/4 [==============================] - 0s 19ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 48/100
4/4 [==============================] - 0s 18ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 49/100
4/4 [==============================] - 0s 24ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 50/100
4/4 [==============================] - 0s 24ms/step - loss: 0.5354 - mse: 0.5354 - ma

4/4 [==============================] - 0s 20ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 90/100
4/4 [==============================] - 0s 20ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 91/100
4/4 [==============================] - 0s 20ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 92/100
4/4 [==============================] - 0s 25ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 93/100
4/4 [==============================] - 0s 19ms/step - loss: 0.5354 - mse: 0.5354 - mae: 0.2727 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.1555
Epoch 94/100
4/4 [==============================] - 0s 20ms/step - loss: 0.5354 - mse: 0.5354 - ma

In [ ]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

In [ ]:
score = model.evaluate(test_x, test_y)
score